## Trabajo Final - Diplomatura en Ciencia de Datos Aplicada 📊
### Integrantes:
* Boladeras Facundo
* Fontanini Pablo
* Miller Marcos
* Baeza Noeli

In [244]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
%matplotlib inline
import matplotlib.gridspec as gridspec
import seaborn as sns
import warnings
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras

# Explorar dataset

In [245]:
# Lectura del csv
df = pd.read_csv(r"Datos_TP_final.csv")

In [246]:
df.head(3)

,CICLO,EVENTO,LOTE,SUP,pp Nov,pp Dic,pp Ene,IPC,EROSION POTENCIAL,EVI 1 Nov media,...,EVI 17 Ene min,EVI 17 Ene max,EVI 2 Feb media,EVI 2 Feb min,EVI 2 Feb max,RINDE,H_Nov,H_Dic,H_Ene,Unnamed: 34
0,2001/02,Neutro,TEMPEL N2,64.8,353,140,156,52,moderadamente susceptible,0.21,...,0.61,0.62,0.49,0.47,0.51,6675,1458,677,1064,NaN
1,2001/02,Neutro,3D,118.4,353,140,156,34,practicamente no susceptible,0.51,...,0.30,0.58,0.19,0.16,0.30,8357,1463,696,1111,NaN
2,2001/02,Neutro,6NORTE,143.6,353,140,156,32,moderadamente susceptible,0.48,...,0.29,0.34,0.19,0.16,0.33,7296,1459,689,1093,NaN


In [247]:
# Visualizar las dimensiones del dataframe
df.shape
print("El dataframe tiene", df.shape[0], "filas y", df.shape[1], "columnas")

El dataframe tiene 176 filas y 35 columnas


In [248]:
# Obtén la columna "RINDE"
rinde_column = df.pop("RINDE")
# Agrega la columna "RINDE" al final del DataFrame
df["RINDE"] = rinde_column
df = df.drop(["Unnamed: 34", "CICLO", "LOTE"], axis=1)
df

,EVENTO,SUP,pp Nov,pp Dic,pp Ene,IPC,EROSION POTENCIAL,EVI 1 Nov media,EVI 1 Nov min,EVI 1 Nov max,...,EVI 17 Ene media,EVI 17 Ene min,EVI 17 Ene max,EVI 2 Feb media,EVI 2 Feb min,EVI 2 Feb max,H_Nov,H_Dic,H_Ene,RINDE
0,Neutro,64.8,353,140,156,52,moderadamente susceptible,0.21,0.20,0.24,...,0.62,0.61,0.62,0.49,0.47,0.51,1458,677,1064,6675
1,Neutro,118.4,353,140,156,34,practicamente no susceptible,0.51,0.46,0.62,...,0.38,0.30,0.58,0.19,0.16,0.30,1463,696,1111,8357
2,Neutro,143.6,353,140,156,32,moderadamente susceptible,0.48,0.46,0.51,...,0.31,0.29,0.34,0.19,0.16,0.33,1459,689,1093,7296
3,Neutro,143.1,358,136,120,32,moderadamente susceptible,0.33,0.25,0.48,...,0.59,0.45,0.76,0.47,0.32,0.60,1455,689,1092,4668
4,Neutro,69.4,337,153,170,50,moderadamente susceptible,0.37,0.24,0.43,...,0.51,0.48,0.53,0.30,0.29,0.34,1483,759,1239,5848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,Niña,92.3,231,367,399,40,moderadamente susceptible,0.34,0.30,0.39,...,0.56,0.53,0.59,0.39,0.34,0.41,783,1315,1483,6484
172,Niña,213.8,228,389,407,50,practicamente no susceptible,0.41,0.35,0.47,...,0.44,0.40,0.53,0.27,0.21,0.47,770,1328,1480,8030
173,Niña,79.1,231,367,399,73,levemente susceptible,0.34,0.32,0.38,...,0.54,0.50,0.68,0.26,0.25,0.28,783,1307,1490,6507
174,Niña,129.1,228,389,407,32,moderadamente susceptible,0.38,0.33,0.52,...,0.58,0.47,0.73,0.37,0.32,0.52,779,1318,1482,7896


In [249]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EVENTO             176 non-null    object 
 1   SUP                176 non-null    float64
 2   pp Nov             176 non-null    object 
 3   pp Dic             176 non-null    object 
 4   pp Ene             176 non-null    object 
 5   IPC                176 non-null    object 
 6   EROSION POTENCIAL  176 non-null    object 
 7   EVI 1 Nov media    176 non-null    float64
 8   EVI 1 Nov min      176 non-null    float64
 9   EVI 1 Nov max      176 non-null    float64
 10  EVI 17 Nov media   176 non-null    float64
 11  EVI 17 Nov min     176 non-null    float64
 12  EVI 17 Nov max     176 non-null    float64
 13  EVI 3 Dic media    176 non-null    float64
 14  EVI 3 Dic min      176 non-null    float64
 15  EVI 3 Dic max      176 non-null    float64
 16  EVI 19 Dic media   176 non

In [250]:
object_columns = df.select_dtypes(include=['object'])

print(object_columns.columns)

Index(['EVENTO', 'pp Nov', 'pp Dic', 'pp Ene', 'IPC', 'EROSION POTENCIAL',
       'H_Nov', 'H_Dic', 'H_Ene'],
      dtype='object')


In [251]:
class ConvertObjectToInt:
    def __init__(self, df, columns_to_convert):
        self.df = df
        self.columns_to_convert = columns_to_convert

    def convert_columns(self):
        for column in self.columns_to_convert:
            if self.df[column].dtype == 'object':
                try:
                    # Intenta convertir valores válidos a numéricos y establecer los no válidos como NaN
                    self.df[column] = pd.to_numeric(self.df[column], errors='coerce')
                    # Rellena los NaN con un valor predeterminado (por ejemplo, 0)
                    self.df[column].fillna(0, inplace=True)
                    # Convierte la columna a tipo int
                    self.df[column] = self.df[column].astype(int)
                except ValueError as e:
                    print(f"Error al convertir la columna {column}: {e}")

In [252]:
columns_to_convert = ["pp Nov", "pp Dic", 'pp Ene', 'H_Nov', 'H_Dic', "H_Ene", "IPC"]

converter = ConvertObjectToInt(df, columns_to_convert)
converter.convert_columns()

# Funciones Aux

In [253]:
def train_val_test_split(df, rstate=42, shuffle=True, stratify=None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size=0.4, random_state=rstate, shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(
        test_set, test_size=0.5, random_state=rstate, shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

def remove_labels(df, label_name):
    X = df.drop(label_name, axis=1)
    y = df[label_name].copy()
    return (X, y)

def evaluate_result(y_pred, y, y_prep_pred, y_prep, metric):
    print(metric.__name__, "Para datos de train:", metric(y_pred, y, average='weighted'))
    print(metric.__name__, "Para datos de validacion:", metric(y_prep_pred, y_prep, average='weighted'))

In [254]:
X = df.drop(columns=['RINDE'])  
y = df['RINDE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [255]:
X_train

,EVENTO,SUP,pp Nov,pp Dic,pp Ene,IPC,EROSION POTENCIAL,EVI 1 Nov media,EVI 1 Nov min,EVI 1 Nov max,...,EVI 1 Ene max,EVI 17 Ene media,EVI 17 Ene min,EVI 17 Ene max,EVI 2 Feb media,EVI 2 Feb min,EVI 2 Feb max,H_Nov,H_Dic,H_Ene
156,Niño,97.7,185,258,130,34,practicamente no susceptible,0.51,0.28,0.64,...,0.73,0.49,0.44,0.62,0.31,0.25,0.46,835,1033,532
136,Neutro,110.3,127,604,91,50,practicamente no susceptible,0.28,0.23,0.34,...,0.69,0.58,0.48,0.77,0.35,0.23,0.53,965,1748,757
98,Niña,79.0,194,56,219,73,levemente susceptible,0.31,0.19,0.37,...,0.25,0.23,0.20,0.28,0.39,0.32,0.50,994,242,786
157,Niño,89.2,173,263,127,40,moderadamente susceptible,0.54,0.49,0.56,...,0.65,0.46,0.41,0.54,0.18,0.14,0.31,815,1110,529
38,Niño,38.0,315,266,304,73,levemente susceptible,0.33,0.30,0.34,...,0.51,0.57,0.52,0.60,0.17,0.16,0.19,1141,1178,1387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Niño,97.7,300,483,228,52,moderadamente susceptible,0.61,0.48,0.66,...,0.62,0.45,0.43,0.47,0.26,0.23,0.37,1068,1633,863
106,Niña,40.0,89,55,226,35,levemente susceptible,0.26,0.24,0.31,...,0.40,0.39,0.36,0.40,0.37,0.36,0.41,865,241,862
14,Niño,46.3,325,409,132,50,moderadamente susceptible,0.34,0.23,0.45,...,0.63,0.50,0.47,0.53,0.39,0.31,0.45,1535,1576,954
92,Niña,169.8,55,164,202,34,practicamente no susceptible,0.29,0.24,0.39,...,0.62,0.30,0.25,0.36,0.22,0.18,0.33,534,616,1023


# Preprosesamiento

In [256]:
class CustomOneHotEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_encode):
        self.columns_to_encode = columns_to_encode
        self._oh = OneHotEncoder(sparse=False)
        self._columns = None

    def fit(self, X, y=None):
        X_cat = X[self.columns_to_encode]
        self._columns = pd.get_dummies(X_cat).columns
        self._oh.fit(X_cat)
        return self

    def transform(self, X, y=None):
        X_copy = X.copy()
        X_cat = X_copy[self.columns_to_encode]
        X_num = X_copy.drop(self.columns_to_encode, axis=1)
        X_cat_oh = self._oh.transform(X_cat)
        X_cat_oh = pd.DataFrame(X_cat_oh,
                                columns=self._columns,
                                index=X_copy.index)
        X_copy = X_num.join(X_cat_oh)
        return X_copy

In [257]:
# Definir las columnas que deseas codificar
columns_to_encode = ['EVENTO', 'EROSION POTENCIAL']

# Crear una instancia de CustomOneHotEncoding
custom_encoder = CustomOneHotEncoding(columns_to_encode)

# A continuación, puedes ajustar y transformar tu DataFrame
X_train_dum = custom_encoder.fit_transform(X_train)
X_test_dum = custom_encoder.fit_transform(X_test)

In [258]:
class CustomScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.robust_scaler = RobustScaler()
        self.robust_scaler.fit(X)
        return self
    
    def transform(self, X, y=None):
        X_scaled = self.robust_scaler.transform(X)
        X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
        return X_scaled_df

In [259]:
custom_scaler = CustomScaler()

X_train_pro = custom_scaler.fit_transform(X_train_dum)
X_test_pro = custom_scaler.fit_transform(X_test_dum)

In [260]:
class PCAReducer:
    def __init__(self, n_components=None):
        self.n_components = n_components
        self.pca = PCA(n_components=self.n_components)
    
    def fit_transform(self, X):
        transformed_data = self.pca.fit_transform(X)
        component_columns = [f"PC{i+1}" for i in range(transformed_data.shape[1])]
        return pd.DataFrame(transformed_data, columns=component_columns)
    
    def explained_variance_ratio(self):
        return self.pca.explained_variance_ratio_
    
    def print_explained_variance_ratio(self):
        variance_ratio = self.explained_variance_ratio()
        for i, ratio in enumerate(variance_ratio):
            print(f"Component {i+1}: {ratio:.2f}")

In [268]:
# Supongamos que df es tu DataFrame
pca_reducer = PCAReducer(n_components=0.90)
x_train_pca= pca_reducer.fit_transform(X_train_pro)
x_test_pca=pca_reducer.fit_transform(X_test_pro)

In [269]:
pca_reducer.print_explained_variance_ratio()

Component 1: 0.25
Component 2: 0.25
Component 3: 0.12
Component 4: 0.08
Component 5: 0.07
Component 6: 0.06
Component 7: 0.04
Component 8: 0.03
Component 9: 0.02


In [272]:
x_test_pca

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9
0,-1.274779,-0.185548,-2.274808,0.070408,1.417499,-0.661746,-1.229992,-0.626932,-0.615250
1,1.395692,-1.125970,-1.731918,0.425338,-1.121296,-0.227170,0.826177,-0.203492,0.066189
2,-0.608878,0.380082,-0.137998,-1.183714,3.464578,1.145433,-0.227946,-0.705373,-0.600207
3,0.061491,-0.608368,1.034561,1.098765,-0.204499,-0.498129,0.278133,-1.095748,0.928579
4,-1.733657,-2.334292,0.729392,-0.599428,-0.569808,-0.650313,-0.109325,-0.121303,-0.964546
5,-0.511813,-2.226270,-0.704347,2.460508,0.785641,-1.353158,-0.221008,-0.280363,-0.736721
6,-0.440869,1.008727,1.588146,1.462652,0.147312,2.498032,-1.365268,0.850691,0.469352
7,1.857881,-0.417614,0.096571,-2.454992,0.812873,0.103504,-0.311268,-1.751749,0.985945
8,2.838869,1.150516,-1.992581,0.680395,0.715567,1.061173,0.680396,0.400366,0.833612
9,-0.961911,0.602820,0.005670,-1.027870,2.121070,0.798493,0.247908,-0.118366,-1.254728


# Modelado

### Regresion Lineal

In [273]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lin_reg = LinearRegression()

lin_reg.fit(x_train_pca, y_train)

LinearRegression()

In [274]:
y_pred = lin_reg.predict(x_test_pca)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 12 is different from 9)

### RF

In [ ]:
# Predicción para el valor mínimo y máximo del conjunto de datos de entrenamiento


### RNN

In [ ]:
model = models.Sequential()

model.add(layers.Dense(1000, activation='relu', input_shape=(X_train_pro.shape[1],)))
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(1, activation='linear'))


optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
              loss='mean_squared_error',  # Cambia a 'mean_squared_error'
              metrics=['mean_absolute_error'])  # Opcional: métrica de error absoluto medio

In [ ]:
history1 = model.fit(X_train_pro,
                   y_train,
                   epochs=50,
                   batch_size=1)                 

Epoch 1/50
140/140 [==============================] - 1s 4ms/step - loss: 53923764.0000 - mean_absolute_error: 6889.5610
Epoch 2/50
140/140 [==============================] - 0s 3ms/step - loss: 49123744.0000 - mean_absolute_error: 6542.8696
Epoch 3/50
140/140 [==============================] - 0s 3ms/step - loss: 27338006.0000 - mean_absolute_error: 4744.5088
Epoch 4/50
140/140 [==============================] - 0s 3ms/step - loss: 7002781.5000 - mean_absolute_error: 2257.5391
Epoch 5/50
140/140 [==============================] - 1s 4ms/step - loss: 3969019.0000 - mean_absolute_error: 1619.0688
Epoch 6/50
140/140 [==============================] - 0s 3ms/step - loss: 3300355.2500 - mean_absolute_error: 1470.7600
Epoch 7/50
140/140 [==============================] - 0s 4ms/step - loss: 2876874.5000 - mean_absolute_error: 1374.9406
Epoch 8/50
140/140 [==============================] - 0s 4ms/step - loss: 2524093.5000 - mean_absolute_error: 1299.0469
Epoch 9/50
140/140 [=================